In [1]:
import gensim
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import os
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [2]:
legal_stopwords = ['a', 'about', 'above', 'according', 'across', 'actually', 'adj', 'after', 'afterwards', 'again',
                   'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'amongst',
                   'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anywhere', 'are', 'aren', 'around',
                   'as', 'at', 'be', 'become', 'because', 'become', 'becomes', 'becoming', 'been', 'before',
                   'beforehand', 'begin', 'beginning', 'behind', 'below', 'beside', 'besides', 'between', 'beyond',
                   'billion', 'both', 'but', 'by', 'can', 'cannot', 'caption', 'co', 'could', 'couldn', 'did', 'didn',
                   'do', 'does', 'doesn', 'don', 'down', 'during', 'each', 'eg', 'eight', 'eighty', 'either', 'else',
                   'elsewhere', 'end', 'ending', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything',
                   'everywhere', 'except', 'few', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty',
                   'found', 'four', 'form', 'further', 'had', 'has', 'hasn', 'have', 'haven', 'he', 'hence', 'her',
                   'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his',
                   'how', 'however', 'hundred', 'ie', 'if', 'in', 'inc', 'Indeed', 'instead', 'into', 'is', 'isn', 'it',
                   'its', 'itself', 'last', 'later', 'latter', 'latterly', 'least', 'less', 'let', 'like', 'likely',
                   'll', 'ltd', 'made', 'make', 'makes', 'many', 'maybe', 'me', 'meantime', 'meanwhile', 'might',
                   'might', 'million', 'miss', 'more', 'moreover', 'most', 'mostly', 'mr', 'mrs', 'much', 'must', 'my',
                   'myself', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'ninety', 'no', 'nobody',
                   'none', 'nonethelesss', 'noon', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of', 'off', 'often',
                   'on', 'once', 'one', 'only', 'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours',
                   'ourselves', 'out', 'over', 'overall', 'own', 'per', 'perhaps', 'rather', 're', 'recent', 'recently',
                   'same', 'seem', 'seemed', 'seeming', 'seems', 'seven', 'seventy', 'several', 'she', 'should',
                   'shouldn', 'since', 'six', 'sixty', 'so', 'some', 'somehow', 'someone', 'sometime', 'sometimes',
                   'somewhere', 'still', 'stop', 'such', 'talking', 'ten', 'than', 'that', 'the', 'their', 'them',
                   'themselves', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'therein',
                   'thereupon', 'these', 'they', 'thirty', 'this', 'those', 'though', 'thousand', 'three', 'through',
                   'throughout', 'thru', 'thus', 'to', 'together', 'too', 'toward', 'towards', 'trillion', 'twenty',
                   'two', 'under', 'unless', 'unlike', 'unlikely', 'until;', 'up', 'upon', 'us', 'used', 'using', 've',
                   'very', 'via', 'very', 'was', 'wasn', 'we', 'well', 'were', 'weren', 'what', 'whatever',
                   'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon',
                   'wherever', 'whether', 'which', 'while', 'whither', 'which', 'while', 'whither', 'who', 'whoever',
                   'whole', 'whom', 'whomever', 'whose', 'why', 'will', 'with', 'within', 'without', 'won', 'would',
                   'wouldn', 'yes', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', 'shall', 'from', 'no.',
                   'ord.', '§', '§§']

In [3]:
def trigram_bow_generator(filepath):
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [4]:
with open("testing.txt") as file:
    documents = list(file)    
    text = [[word for word in document.lower().split() if word not in legal_stopwords]for document in documents]
    with open("Output.txt", 'w') as output_file:
        for line in text:
            output_file.write(' '.join(line))
            output_file.write('\n')

In [5]:
trigram_corpus = LineSentence("CorpusChristi.txt")
trigram_dictionary = Dictionary(trigram_corpus)
trigram_dictionary.filter_extremes(no_below=20, no_above=0.4)
trigram_dictionary.compactify()
MmCorpus.serialize("check", trigram_bow_generator("CorpusChristi.txt"))
trigram_bow_corpus = MmCorpus("check")

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # workers => sets the parallelism, and should be
    # set to your number of physical cores minus one
    lda = LdaMulticore(trigram_bow_corpus, num_topics=12,
                       id2word=trigram_dictionary,
                       workers=3)

In [7]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, trigram_dictionary)
pyLDAvis.display(LDAvis_prepared)

/home/jackson/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
